# Quantum Fourier Transform I

Implementation of $QFT$ and $QFT^+$ gates using __[Cirq](https://quantumai.google/cirq)__. The implementation is then tested by applying the operator $QFTQFT^{+}$ on the state $\left| \psi \right\rangle = \left| 100 \right\rangle$.

The $n$-cubit $QFT$ gate implemented in this example is given by <a name="ref-1"/>[(Nielsen and Chuang, 2010)](#cite-MikeIke)

\begin{equation}
QFT = \frac{1}{\sqrt{2^n}}\sum_{j=0}^{2^n-1}\sum_{k=0}^{2^n-1}e^{2\pi ikj/{2^n}}\left| k \right\rangle \left\langle j \right|
\end{equation}


In [10]:
#QFT test. Testing QFTD*QFT|state> = |state>
#state = |100>

In [2]:
import random
import numpy as np
import cirq
import statistics
from statistics import mode
from cirq import H, X, CNOT, S,measure

In [3]:
#QFT gate
#Inputs: quantum circuit and it's qubits
#Output: quantum circuit with QFT applied to first nq qubits
def QFT(qc,qbts,nq):
    
    if nq == 1:
       n = len(qbts) #Number of qubits
    else:
       n = nq #Apply to first nq qubits
    
    #Apply S gates with positive exponents
    for i in range(n):
        qc.append(H(qbts[i])) #Add H gate
        for j in range(i+1,n):
            po = j-i-1
            exp = 1.0/(2**po)
            SS = S**exp
            qc.append(SS(qbts[i]).controlled_by(qbts[j]))
    
    #Swap the qubits
    for i in range(int(n/2)):
        qc.append(cirq.SWAP(qbts[i],qbts[n-1-i]))
        
    return qc

In [4]:
#QFT+ gate
#Inputs: quantum circuit and it's qubits
#Output: quantum circuit with QFT+ applied
def QFTD(qc,qbts,nq):
    if nq == 1:
        n = len(qbts) #Number of qubits
    else:
        n = nq #Apply to first nq qubits
        
    #Apply S gates with negative exponents
    for i in range(n):
        qc.append(H(qbts[i])) #Add H gate
        for j in range(i+1,n):
            po = j-i-1
            exp = -1.0/(2**po)
            SS = S**exp
            qc.append(SS(qbts[i]).controlled_by(qbts[j]))
            
            
    #Swap the qubits
    for i in range(int(n/2)):
        qc.append(cirq.SWAP(qbts[i],qbts[n-1-i]))
        
    return qc

In [5]:
N = 3 #Number of qubits

#Prepare |100>
state = [1,0,0] #Initial state
qbs = cirq.LineQubit.range(N)
qc = cirq.Circuit()
qc.append(X(qbs[0]))
        
#print("Circuit:")
#print(qc)

In [6]:
#Apply QFT 
qc = QFT(qc,qbs,N)

print("Circuit:")
print(qc)

Circuit:
                  ┌──┐
0: ───X───H───S────T─────────────×───
              │    │             │
1: ───────────@────┼H────S───────┼───
                   │     │       │
2: ────────────────@─────@───H───×───
                  └──┘


In [7]:
#Final state vector before measurement
simulator = cirq.Simulator()
Result = simulator.simulate(qc, qubit_order=[qbs[0], qbs[1],qbs[2]])
State1 = np.around(Result.final_state_vector, 8)
print(State1)

[ 0.35355335+0.j -0.35355335+0.j  0.35355335+0.j -0.35355335+0.j
  0.35355335+0.j -0.35355335+0.j  0.35355335+0.j -0.35355335+0.j]


In [8]:
#Apply QFT+ to state 1 above
Ns = 2**N
Final_State = np.zeros(Ns,dtype=complex) #Final state

count = 0
for i in range(0,2):
    for j in range(0,2):
        for k in range(0,2):
            
            #Quantum circuit
            qbs1 = cirq.LineQubit.range(N)
            qc1 = cirq.Circuit()
            if i == 1:
                qc1.append(X(qbs1[0]))
                
            if j == 1:
                qc1.append(X(qbs1[1]))
                
            if k == 1:
                qc1.append(X(qbs1[2]))
                
            #Apply QFT+ 
            qc1 = QFTD(qc1,qbs1,N)
            
            #Get state vector before measurement
            Result = simulator.simulate(qc1, qubit_order=[qbs1[0], qbs1[1],qbs1[2]])
            State = np.around(Result.final_state_vector, 8)
            
            #Add state to final state with appropriate multiplier
            Final_State += State1[count]*State
            
            count+=1

In [9]:
#Final state should be |100>
print(Final_State)

[0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.99999982+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


<!--bibtex

@book{MikeIke,
    title = {Quantum Computation and Quantum Information},
    author = {Nielsen, Michael A. and Chuang, Issac L},
    year = {2010},
    publisher = {Cambridge University Press}
}

-->

# References

<a name="cite-MikeIke"/><sup>[^](#ref-1) </sup>Nielsen, Michael A. and Chuang, Issac L. 2010. _Quantum Computation and Quantum Information_.

